In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
messages = pd.read_csv('..\data\messages.csv')
print(len(messages))
messages.head()

26248


,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
categories = pd.read_csv('..\data\categories.csv')
print(len(categories))
categories.head()

26248


,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [4]:
df = messages.merge(on = 'id', right = categories)

row = df['categories'][0]
categories_column_names = row.split(';')
categories_column_names= [category.split('-')[0] for category in categories_column_names]
rename_dict = {}
for index, category in enumerate(categories_column_names):
    rename_dict[index] = category
    
df = df.join(df['categories'].str.split(';', expand=True).rename(columns = rename_dict))
df = df.drop(columns = ['categories'])

for col_name in categories_column_names:
    df[col_name] = df[col_name].apply(lambda x: x.split('-')[1])
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
for col_name in categories_column_names:
    if df[col_name].nunique() != 2:
        print(f'{col_name} - {df[col_name].unique()}')

related - ['1' '0' '2']
child_alone - ['0']


In [6]:
df['related'].value_counts()

1    20042
0     6140
2      204
Name: related, dtype: int64

In [7]:
df = df[df['related'] != '2']
df['related'].value_counts()

1    20042
0     6140
Name: related, dtype: int64

In [8]:
df = df.drop_duplicates(subset = ['id'], keep = False)
print(f'len df = {len(df)}')
print(f'df unique ids = {len(df.id.unique())}')
print(f'df unique messages = {len(df.message.unique())}')

df = df.drop_duplicates(subset = ['message'], keep = False)
print(f'len df = {len(df)}')
print(f'df unique ids = {len(df.id.unique())}')
print(f'df unique messages = {len(df.message.unique())}')

len df = 25931
df unique ids = 25931
df unique messages = 25928
len df = 25927
df unique ids = 25927
df unique messages = 25927


In [9]:
df2 = df.groupby(['id'], as_index = False).count()
dup = list(df2[df2['message'] != 1]['id'])
df = df[~df['id'].isin(dup)]
len(df.id.unique())

25927

In [10]:
len(df)

25927

In [11]:
len(df.message.unique())

25927

In [12]:
len(df.id.unique())

25927

In [13]:
df = df.drop_duplicates()

In [14]:
df1 = df.groupby(['message'], as_index = False).nunique()
df1[df1['id'] != 1]

,message,id,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


In [15]:
df = df[df['message'] != '#NAME?']
len(df.id.unique())

25927

In [16]:
len(df.message.unique())

25927

In [17]:
len(df)

25927

In [18]:
df2 = df.groupby(['id'], as_index = False).count()
dup = list(df2[df2['message'] != 1]['id'])
df = df[~df['id'].isin(dup)]
len(df.id.unique())

25927

*25954

In [19]:
len(df.message.unique())

25927

In [20]:
len(df)

25927

In [21]:
engine = create_engine('sqlite:///disaster.db')
df.to_sql('disaster_response', engine, index=False)

ValueError: Table 'disaster_response' already exists.